<a href="https://colab.research.google.com/github/joseorlandomx/topicodeindustria1/blob/main/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Final (Dashboard tema libre)**

José Orlando Salas Contreras (208743738)

[jose.salas4373@alumnos.udg.mx](mailto:jose.salas4373@alumnos.udg.mx)

Verónica Marisela Frías Peralta (208561486)

[veronica.frias6148@alumnos.udg.mx](mailto:veronica.frias6148@alumnos.udg.mx)

**Requisitos mínimos:**
* **Capa 1**: Un panel de controles que permita generar el **dataset origen** a analizar. Este dataset puede ser generado al momento partiendo de una formula e introduciendo los parametros a travez de los widgets en el panel (**ejemplo**: selección del tipo de trayectoria y sus parámetros en la práctica 4), o puede ser el resultado de filtrar un dataset mayor ya existente en memoria, en cuyo caso los widgets en el panel de control permitiran ajustar los limites de dichos filtros.
* **Capa 2**: Un panel en el cual se despliegue una **representación gráfica del dataset origen** (**ejemplo**: Trazo de la trayectoria en 3D en la práctica 4).
* **Capa 3**: Un panel de controles que permita elegir entre diferentes **metricas** a calcular a partir del dataset origen. Las métricas obtenidas deberán ser representadas también graficamente (**ejemplo**: Selección entre path length y mean squared displacement y su gráfica en la práctica 4).

**Revisión de la práctica:**
* El notebook con la práctica se entregará en un repositorio en **GitHub**. Será necesario demostrar por medio de commits el historial de versiones de su NoteBook.
* Los integrantes del equipo deberán explicar su código y deberán ser capaces de corregir errores introducidos a este.
* Uno de los integrantes del equipo presentará el dashboard durante la clase.

## **Modules**

In [139]:
# Importar bibliotecas necesarias
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import panel as pn
import panel.widgets as pnw
import plotly.express as px
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

# Extensión para visualización interactiva en Panel
pn.extension('plotly')

# Descomentar si se está ejecutando en Google Colab
# from google.colab import drive
# drive.mount('/content/gdrive/')

# Importar widgets de IPython
import ipywidgets as widgets

# Importar Panel nuevamente (puede haber sido importado previamente, pero está duplicado aquí)
import panel as pn
import panel.widgets as pnw

In [140]:
# Cargar el conjunto de datos
url = '/content/conjunto_de_datos_gastotarjetas_enigh2022_ns.csv'
df_origen = pd.read_csv(url)
df_auxiliar = pd.read_csv(url)

# Crear un diccionario de categorías para mapear claves a descripciones
categorias = {
    'TB01': 'Alimentos, bebidas y tabaco',
    'TB02': 'Artículos o servicios destinados a la limpieza y cuidados de la casa',
    'TB03': 'Artículos o servicios destinados a cuidados personales',
    'TB04': 'Artículos o servicios destinados a educación, cultura y recreación',
    'TB05': 'Artículos o servicios destinados a la comunicación y servicios para vehículos',
    'TB06': 'Artículos o servicios destinados a vivienda y servicios de conservación',
    'TB07': 'Último recibo pagado',
    'TB08': 'Prendas de vestir, calzado y accesorios',
    'TB09': 'Cristalería, blancos y utensilios domésticos',
    'TB10': 'Artículos o servicios destinados a cuidados de la salud',
    'TB11': 'Enseres domésticos y mantenimiento de la vivienda',
    'TB12': 'Artículos de esparcimiento',
    'TB13': 'Artículos o servicios destinados al transporte',
    'TB14': 'Gastos diversos',
    'TB15': 'Erogaciones financieras y de capital',
    'TR01': 'Alimentos, bebidas y tabaco',
    'TR02': 'Artículos o servicios destinados a la limpieza y cuidados de la casa',
    'TR03': 'Artículos o servicios destinados a cuidados personales',
    'TR04': 'Artículos o servicios destinados a educación, cultura y recreación',
    'TR05': 'Artículos o servicios destinados a la comunicación y servicios para vehículos',
    'TR06': 'Artículos o servicios destinados a vivienda y servicios de conservación',
    'TR07': 'Último recibo pagado',
    'TR08': 'Prendas de vestir, calzado y accesorios',
    'TR09': 'Cristalería, blancos y utensilios domésticos',
    'TR10': 'Artículos o servicios destinados a cuidados de la salud',
    'TR11': 'Enseres domésticos y mantenimiento de la vivienda',
    'TR12': 'Artículos de esparcimiento',
    'TR13': 'Artículos o servicios destinados al transporte',
    'TR14': 'Gastos diversos',
    'TR15': 'Erogaciones financieras y de capital',
}


## **Capa 1**


In [152]:
# Mapear las claves a descripciones en el DataFrame
df['descripcion'] = df['clave'].map(categorias)

# Crear menú desplegable para seleccionar categorías
categoria_selector = pnw.CrossSelector(name='Seleccione Categorías', options=list(categorias.values()))

# Crear controles deslizantes para el rango de gasto
rango_gasto_selector = pnw.RangeSlider(name='Rango de Gasto', start=100, end=50000, step=100, value=(100, 50000))

# Crear gráfico dinámico inicial (sin filtrar)
fig_inicial = px.scatter_3d(df, x='descripcion', y='foliohog', z='gasto',
                            color='gasto', opacity=0.7, size='gasto', size_max=30,
                            category_orders={'descripcion': list(categorias.values())},
                            labels={'descripcion': 'Categoría', 'foliohog': 'Número de Hogar', 'gasto': 'Gasto'},
                            title='Gastos en 3D por Categoría y Hogar')

# Crear gráfica dinámica con el gráfico inicial
grafica_dinamica = pn.pane.Plotly(fig_inicial)

# Función para filtrar el conjunto de datos y actualizar la gráfica
def filtrar_datos(event):
    categorias_seleccionadas = categoria_selector.value
    rango_gasto = rango_gasto_selector.value

    # Aplicar filtros al DataFrame
    df_filtrado = df[df['descripcion'].isin(categorias_seleccionadas) & (df['gasto'].between(*rango_gasto))]

    # Crear el gráfico 3D de barras
    fig = px.scatter_3d(df_filtrado, x='descripcion', y='foliohog', z='gasto',
                        color='gasto', opacity=0.7, size='gasto', size_max=30,
                        category_orders={'descripcion': list(categorias.values())},
                        labels={'descripcion': 'Categoría', 'foliohog': 'Número de Hogar', 'gasto': 'Gasto'},
                        title='Gastos Filtrados en 3D por Categoría y Hogar')

    # Actualizar la gráfica dinámica
    grafica_dinamica.object = fig

# Vincular la función de filtrado al cambio en los controles
categoria_selector.param.watch(filtrar_datos, 'value')
rango_gasto_selector.param.watch(filtrar_datos, 'value')

# Mostrar el panel con los controles y la gráfica dinámica
pn.Column(categoria_selector, rango_gasto_selector, grafica_dinamica).servable()

Column
    [0] CrossSelector(name='Seleccione Categorías', options=['Alimentos, bebidas y tab...])
    [1] RangeSlider(end=50000, name='Rango de Gasto', start=100, step=100, value=(100, 50000), value_end=50000, value_start=100)
    [2] Plotly(Figure)

## **Capa 2**

In [142]:
# Mapear las claves a descripciones en el DataFrame
df['descripcion'] = df['clave'].map(categorias)

# Crear el gráfico 3D de barras
fig = px.scatter_3d(df, x='descripcion', y='foliohog', z='gasto',
                    color='gasto', opacity=0.7, size='gasto', size_max=30,
                    category_orders={'descripcion': list(categorias.values())},
                    labels={'descripcion': 'Categoría', 'foliohog': 'Número de Hogar', 'gasto': 'Gasto'},
                    title='Dataset Original de Gastos Categoría y Hogar')

# Mostrar el gráfico en un panel
pn.Row(fig).servable()


Row
    [0] Plotly(Figure)

## **Capa 3**

In [151]:
# Mapear las claves a descripciones en el DataFrame
df['descripcion'] = df['clave'].map(categorias)

# Lista de métricas disponibles
metricas_disponibles = ['Media de Gastos por Categoría y Hogar', 'Desviación Estándar de Gastos por Categoría y Hogar']

# Crear menú desplegable para seleccionar la métrica
metrica_selector = pnw.Select(name='Seleccione la métrica:', options=metricas_disponibles)

# Resultado container
resultado_container = pn.Column()

# Función para calcular y mostrar la métrica seleccionada
def calcular_metrica(event):
    # Limpiar el contenedor de resultados
    resultado_container.clear()

    metrica = metrica_selector.value

    if metrica == 'Media de Gastos por Categoría y Hogar':
        # Calcular la media de gastos por categoría y hogar
        media_gastos = df.groupby(['descripcion', 'foliohog'])['gasto'].mean().reset_index()

        # Gráfico de barras para la media de gastos
        fig_media = px.bar(media_gastos, x='descripcion', y='gasto', color='foliohog',
                           labels={'descripcion': 'Categoría', 'gasto': 'Media de Gastos'},
                           title='Media de Gastos por Categoría y Hogar')

        # Ajustar el tamaño y rotar las etiquetas del eje x para mejorar la visualización
        fig_media.update_layout(
            xaxis=dict(tickangle=-45),
            height=600,  # Ajustar el tamaño según sea necesario
            width=620,   # Establecer el ancho de la gráfica
        )

        resultado_container.append(fig_media)

    elif metrica == 'Desviación Estándar de Gastos por Categoría y Hogar':
        # Calcular la desviación estándar de gastos por categoría y hogar
        std_gastos = df.groupby(['descripcion', 'foliohog'])['gasto'].std().reset_index()

        # Gráfico de barras de error para la desviación estándar de gastos
        fig_std = px.bar(std_gastos, x='descripcion', y='gasto', color='foliohog', error_y='gasto',
                         labels={'descripcion': 'Categoría', 'gasto': 'Desviación Estándar de Gastos'},
                         title='Desviación Estándar de Gastos por Categoría y Hogar')

        # Establecer el ancho de la gráfica
        fig_std.update_layout(
            width=620,   # Establecer el ancho de la gráfica
        )

        resultado_container.append(fig_std)

# Vincular la función de cálculo al cambio en el selector de métricas
metrica_selector.param.watch(calcular_metrica, 'value')

# Organizar visualmente los elementos en el panel
layout = pn.Column(
    '# Análisis de Gastos',
    metrica_selector,
    resultado_container
)

# Mostrar el panel
layout.servable()

Column
    [0] Markdown(str)
    [1] Select(name='Seleccione la métrica:', options=['Media de Gastos por Cate...], value='Media de Gastos p...)
    [2] Column()